In [67]:
import pandas as pd
import numpy as np
from my_utils import *

In [68]:
# initializing filenames to read data which is downloaded from "seffaflik.epias.com.tr" 
ptfName = "PTF-01012016-08122018.csv" # day-ahead (ptf) prices
smfName = "SMF-01012016-07122018.csv" # System Marginal Price (smf) prices
yukTahminName = "YukTahminPlani-01012016-07122018.csv" # load forecast
yuk_gercekName = "GercekZamanliTuketim-01012016-07122018.csv" # real-time consumption

# initializing paths
dataPath = "downloadedData/" # path of downloaded files
savePath = "dataset/" # path of the file to be generated

# initializing dates ...
tarih = "07/12/2018"

# initializing filename to be saved
filename = savePath+"-".join(tarih.split("/"))+".csv"

In [69]:
# reading day-ahead (ptf) data
df = pd.read_csv(dataPath+ptfName, decimal=',')
df.columns = ["tarih", "saat", "ptf"]
df.ptf = floataCevir(df.ptf)
df.ptf  = df.ptf.astype(np.float64)

# generating day feature
df["gun"] = df.apply(lambda x: weekDay(x.tarih.split("/")), axis=1)
# generating month feature
df["ay"] = df.apply(lambda x: int(x.tarih.split("/")[1]), axis=1)
# generating year feature
df["yil"] = df.apply(lambda x: int(x.tarih.split("/")[2]), axis=1)
# generating lagged day-ahead (ptf) prices
for l in [1, 23, 24, 48, 72, 144, 168, 312, 336]:
    df['ptf_l{}'.format(l)] = df.ptf.shift(l)
# generating next day day-ahead (ptf) prices
df['ptf_n24'] = df.ptf.shift(-24)

df.to_csv(filename)
print("day-ahead (ptf) prices features generated.")

day-ahead (ptf) prices features generated.


In [70]:
# reading dataset
df = pd.read_csv(filename)

# reading System Marginal Price (smf) data
smf = pd.read_csv(dataPath+smfName, decimal=',')
smf.columns = ["tarih", "saat", "smf"]
smf.smf = floataCevir(smf.smf)

# generating 24 and 144 hour lagged System Marginal Price (smf) prices
smf['smf_l24'] = smf.smf.shift(24)
smf['smf_l144'] = smf.smf.shift(144)

# merging with dataset
df = pd.merge(df, smf[["smf_l24","smf_l144","tarih","saat"]], how='left', on=["tarih","saat"])

df.iloc[:-24, :].to_csv(filename)
print("System Marginal Price (smf) prices features generated.")

System Marginal Price (smf) prices features generated.


In [71]:
# reading load forecast
df = pd.read_csv(filename)
yuk_tahmin = pd.read_csv(dataPath+yukTahminName, decimal=',', skiprows=1, header=None)
yuk_tahmin.columns = ["tarih", "saat", "yuk_tahmin_l24"]
yuk_tahmin.yuk_tahmin_l24 = floataCevir(yuk_tahmin.yuk_tahmin_l24)

# generating 144 hours lagged load forecast
yuk_tahmin['yuk_tahmin_l{}'.format(144)] = yuk_tahmin.yuk_tahmin_l24.shift(144)

# merging with dataset
df = pd.merge(df, yuk_tahmin[["yuk_tahmin_l144","yuk_tahmin_l24","tarih","saat"]], how='left', on=["tarih","saat"])

df.to_csv(filename)


In [72]:
df = pd.read_csv(filename)

# reading real-time consumption
yuk_gercek = pd.read_csv(dataPath+yuk_gercekName, decimal=',', skiprows=1, header=None)
yuk_gercek.columns = ["tarih", "saat", "yuk_gercek"]

# fixing date and real-time consumption values
yuk_gercek.tarih = yuk_gercek.tarih.str.replace(".", "/")
yuk_gercek.yuk_gercek = floataCevir(yuk_gercek.yuk_gercek)

# generating 24 and 144 hours lagged real-time consumption
for l in [24]:
    yuk_gercek['yuk_gercek_l{}'.format(l)] = yuk_gercek.yuk_gercek.shift(l)
for l in [144]:
    yuk_gercek['yuk_gercek_l{}'.format(l)] = yuk_gercek.yuk_gercek.shift(l)

# merging with dataset
df = pd.merge(df, yuk_gercek[["yuk_gercek_l24","yuk_gercek_l144","tarih","saat"]], how='left', on=["tarih","saat"])

df.to_csv(filename)


In [73]:
df = pd.read_csv(filename)

# reading data downloaded from "rp5.ru"
sicaklik1 = pd.read_csv(dataPath+"temperatures/ankara.csv")
sicaklik1["tarih"] = sicaklik1.apply(lambda x: x.zaman.split(" ")[0], axis=1)
sicaklik1["saat"] = sicaklik1.apply(lambda x: x.zaman.split(" ")[1], axis=1)
sicaklik1.tarih = sicaklik1.tarih.str.replace('.', '/')

for l in [24]:
    sicaklik1['ankaraT_n{}'.format(l)] = sicaklik1.ankaraT.shift(-l)

# merging with dataset
df = pd.merge(df, sicaklik1[["ankaraT_n24","tarih","saat"]], how='left', on=["tarih","saat"])

df.to_csv(filename)


In [74]:
df = pd.read_csv(filename)

# reading data downloaded from "rp5.ru"
sicaklik1 = pd.read_csv(dataPath+"/temperatures/istanbul.csv")
sicaklik1["tarih"] = sicaklik1.apply(lambda x: x.zaman.split(" ")[0], axis=1)
sicaklik1["saat"] = sicaklik1.apply(lambda x: x.zaman.split(" ")[1], axis=1)
sicaklik1.tarih = sicaklik1.tarih.str.replace('.', '/')

for l in [24]:
    sicaklik1['istanbulT_n{}'.format(l)] = sicaklik1.istanbulT.shift(-l)

# merging with dataset
df = pd.merge(df, sicaklik1[["istanbulT_n24","tarih","saat"]], how='left', on=["tarih","saat"])
df.istanbulT_n24.ffill(inplace=True)

df.to_csv(filename)


In [75]:
df = pd.read_csv(filename)

# generating numerical feature of day and hour values 
df["gunS"] = df.apply(lambda x: weekDayNum(x.gun), axis=1)
df["saatS"] = df.apply(lambda x: int(x.saat.split(":")[0]), axis=1)

df.iloc[408:-24, :].loc[:, "tarih":].to_csv(filename)

In [76]:
df = pd.read_csv(filename)

# filling missing temperature values
df.ankaraT_n24.ffill(inplace=True)
# generating avarage temperature
df["sicaklik_ort"] = (df.ankaraT_n24.astype(float) + df.istanbulT_n24.astype(float) )/2

df.loc[:, "tarih":].to_csv(filename, index=False)

In [77]:
df = pd.read_csv(filename)
df.tail()

,tarih,saat,ptf,gun,ay,yil,ptf_l1,ptf_l23,ptf_l24,ptf_l48,...,smf_l144,yuk_tahmin_l144,yuk_tahmin_l24,yuk_gercek_l24,yuk_gercek_l144,ankaraT_n24,istanbulT_n24,gunS,saatS,sicaklik_ort
25305,06/12/2018,19:00,285.44,Cuma,12,2018,290.47,299.22,174.00,302.21,...,274.88,37300.0,38300.0,38020.11,37940.11,9.5,27.0,5,19,18.25
25306,06/12/2018,20:00,286.24,Cuma,12,2018,285.44,303.18,299.22,300.37,...,196.18,36600.0,37400.0,37359.11,36999.68,9.3,27.0,5,20,18.15
25307,06/12/2018,21:00,284.50,Cuma,12,2018,286.24,280.98,303.18,299.22,...,196.18,35700.0,36200.0,36428.37,36031.98,9.1,27.0,5,21,18.05
25308,06/12/2018,22:00,271.99,Cuma,12,2018,284.50,173.99,280.98,264.34,...,190.00,34800.0,35600.0,35394.00,35224.00,8.5,27.0,5,22,17.75
25309,06/12/2018,23:00,251.62,Cuma,12,2018,271.99,280.20,173.99,262.90,...,222.18,33700.0,33900.0,33716.75,33752.94,8.4,27.0,5,23,17.70


In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25310 entries, 0 to 25309
Data columns (total 27 columns):
tarih              25310 non-null object
saat               25310 non-null object
ptf                25310 non-null float64
gun                25310 non-null object
ay                 25310 non-null int64
yil                25310 non-null int64
ptf_l1             25310 non-null float64
ptf_l23            25310 non-null float64
ptf_l24            25310 non-null float64
ptf_l48            25310 non-null float64
ptf_l72            25310 non-null float64
ptf_l144           25310 non-null float64
ptf_l168           25310 non-null float64
ptf_l312           25310 non-null float64
ptf_l336           25310 non-null float64
ptf_n24            25310 non-null float64
smf_l24            25310 non-null float64
smf_l144           25310 non-null float64
yuk_tahmin_l144    25310 non-null float64
yuk_tahmin_l24     25310 non-null float64
yuk_gercek_l24     25310 non-null float64
yuk_gercek_l144 

In [79]:
df.describe()

,ptf,ay,yil,ptf_l1,ptf_l23,ptf_l24,ptf_l48,ptf_l72,ptf_l144,ptf_l168,...,smf_l144,yuk_tahmin_l144,yuk_tahmin_l24,yuk_gercek_l24,yuk_gercek_l144,ankaraT_n24,istanbulT_n24,gunS,saatS,sicaklik_ort
count,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,...,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000,25310.000000
mean,177.653158,6.479099,2016.990913,177.696178,177.489721,177.501080,177.350460,177.137769,176.713878,176.570991,...,168.835222,32302.297511,32300.432635,32558.704730,32553.606647,13.878214,18.493915,3.998815,11.495891,16.186065
std,72.315391,3.341439,0.808601,72.254572,72.329902,72.292572,72.242322,72.275719,72.022814,71.945552,...,80.221788,5557.657826,5606.598851,4981.606481,4980.116843,9.511340,8.345915,1.999071,6.922609,7.291668
min,0.000000,1.000000,2016.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,-15.200000,0.000000,1.000000,0.000000,-5.500000
25%,135.000000,4.000000,2016.000000,135.000000,135.000000,135.000000,134.990000,134.990000,134.960000,134.960000,...,119.990000,28400.000000,28400.000000,28603.497500,28603.497500,6.600000,12.000000,2.000000,5.000000,11.050000
50%,171.675000,6.000000,2017.000000,171.720000,171.190000,171.185000,171.100000,170.965000,170.340000,170.110000,...,163.680000,32700.000000,32700.000000,32812.630000,32814.110000,13.800000,22.000000,4.000000,11.000000,16.500000
75%,209.627500,9.000000,2018.000000,209.617500,209.500000,209.500000,209.490000,209.437500,209.010000,208.990000,...,210.000000,35900.000000,35900.000000,35950.297500,35931.167500,21.000000,25.000000,6.000000,17.000000,21.950000
max,1899.990000,12.000000,2018.000000,1899.990000,1899.990000,1899.990000,1899.990000,1899.990000,1899.990000,1899.990000,...,1899.990000,46500.000000,46500.000000,47062.400000,47062.400000,38.100000,29.000000,7.000000,23.000000,31.950000
